# Camera Navigation

Last year we started to work on camera navigation last year when we had the
limelight target on the shooting goal. We toyed with the iead that we could use this
to compute our field position and use that as position feedback for path following. We
did not get very far on that, so it seemed like now is a good opportunity to learn
more about that.

Our camera has some interesting aspects that make the math around navigation a
bit challenging. We will get to that in a bit. We will start by defining some constants
that locate the camera relative to the robot. Specifically, we use the coordinate system
that (0,0,0) is on the ground under the center of the robot; +X is toward the right,
+Y is towards the front, and +Z is up:

In [2]:
/* The Z position (relative to the robot) of the limelight lens in meters. */
double LIMELIGHT_HEIGHT = .52;
/* The elevation angle of the camera relative to the XY plane (the floor) in radians. */
double LIMELIGHT_ELEV_ANGLE = 0.337;
/* The X position (relative to the robot) of the limelight lens in meters. */
double LIMELIGHT_X_OFFSET = -0.20;
/* The Y position (relative to the robot) of the limelight lens in meters. */
double LIMELIGHT_Y_OFFSET = 0.14;

/* The hieght of the target above the floor */
double TARGET_HEIGHT = 1.205;

// Just print out a little info here for later reference
double toInches(double meters) {
    return meters * 39.3701;
}

System.out.println("LIMELIGHT_HEIGHT = " + LIMELIGHT_HEIGHT + "m (" + toInches(LIMELIGHT_HEIGHT) + "\")");
System.out.println("LIMELIGHT_ELEV_ANGLE = " + LIMELIGHT_ELEV_ANGLE + "rad (" +
                   Math.toDegrees(LIMELIGHT_ELEV_ANGLE) + "deg)");
System.out.println("LIMELIGHT_X_OFFSET = " + LIMELIGHT_X_OFFSET + "m (" + toInches(LIMELIGHT_X_OFFSET) + "\")");
System.out.println("LIMELIGHT_Y_OFFSET = " + LIMELIGHT_Y_OFFSET + "m (" + toInches(LIMELIGHT_Y_OFFSET) + "\")");
System.out.println("TARGET_HEIGHT = " + TARGET_HEIGHT + "m (" + toInches(TARGET_HEIGHT) + "\")");


LIMELIGHT_HEIGHT = 0.52m (20.472452")
LIMELIGHT_ELEV_ANGLE = 0.337rad (19.308677695908745deg)
LIMELIGHT_X_OFFSET = -0.2m (-7.874020000000001")
LIMELIGHT_Y_OFFSET = 0.14m (5.511814")
TARGET_HEIGHT = 1.205m (47.440970500000006")


We calibrate the elevation angle by moving the robot until the target is directly centered
on the limelight, then we measure the horizontal distance (parallel to the floor), and the actual distance between the lens and the center of the target. We use the pythagorean theorem to make sure we have these measurements close to a right triangle; and then use
the `Math.atan2()` method to compute the `LIMELIGHT_ELEV_ANGLE`.

The geometry looks like this:
![alt text](./resources/limelight_side_geom.jpg "limelight side view")

In [3]:
// method to compute the hypotenuse of a right triangle
double computeHypotenuse(double a, double b)
{
    return Math.sqrt((a*a) + (b*b));
}

// the measured target distance (parallel to the floor)
double target_distance = 1.955;
// the measured limelight lens to target distance
double lens_to_target_distance = 2.07;

// compute the hypotenuse from the measuers geometry
computeHypotenuse(TARGET_HEIGHT - LIMELIGHT_HEIGHT, target_distance);

2.0715332485866598

OK, this is really good. The computed hypotenuse is almost exactly equal to our measured hypotenuse
`lens_to_target_distance = 2.07`, so now that we have the geometry that puts the target exactly in the
center of the limelight image we can compute the `LIMELIGHT_ELEV_ANGLE`:

In [4]:
double elev = Math.atan2(TARGET_HEIGHT - LIMELIGHT_HEIGHT, target_distance);
System.out.println("limelight elevation in radians = " + elev);
System.out.println("limelight elevation in degrees = " + Math.toDegrees(elev));

limelight elevation in radians = 0.33701654394342206
limelight elevation in degrees = 19.30962559404333


So this gives us the `LIMELIGHT_ELEV_ANGLE` that is set in the constants. This is a far more accurate method
to determine the elevation angle than trying to physically measure it with a protractor.

## Simple Distance from Limelight to Target

Now that we have our constants in place we need to veryify the accuracy by doing a little testing by moving the robot and computing the distance using the Y angle reported by the limelight. Let's add a bit to the geometry diagram to explain this:
![alt text](./resources/simple_distance.jpg "simple distance with limelight")

We know `LIMELIGHT_ELEV_ANGLE`, the limelight reports the `target_delta`, and

` a = TARGET_HEIGHT - LIMELIGHT_HEIGHT`

which lets us comute the `b` or `target_distance` as `a / Math.tan(LIMELIGHT_ELEV + new_target_delta);`

So let's look at a new position with a `target_delta` of 7.26&deg;, which we need to convert to radians:

In [6]:
double target_delta = Math.toRadians(7.26);
(TARGET_HEIGHT - LIMELIGHT_HEIGHT) / Math.tan(LIMELIGHT_ELEV_ANGLE + target_delta);

1.3697832431119432

And, this is very close (within 1cm) to the distance we measure.

So, let's turn this into a method that returns distance as a function of `target_delta`, verify that it works for this position, and the try a new position:

In [7]:
double getTargetDistance(double target_delta)
{
    return (TARGET_HEIGHT - LIMELIGHT_HEIGHT) / Math.tan(LIMELIGHT_ELEV_ANGLE + target_delta);
}

target_distance = getTargetDistance(target_delta);
target_distance

1.3697832431119432

This is good, the `get_target_distance()` is returning the correct value. Let's reposition the robot for
a second verification - in this case with a `target_delta` of 13.98&deg;:

In [8]:
double new_target_delta = Math.toRadians(13.98);
target_distance = getTargetDistance(new_target_delta);
target_distance

1.043262050559926

Which again is very close to the measured value!!! Yay, progress.

## Where Are We Going With This?

We have initial success in accurately computing distance to the target when the camera is centered on the target,
what's next? What we really want to know is whether we can use a target (or targets), with known position(s) and
height(s) on the field, to compute the the robot position on the field.

Let's diagram this. The first thing is that we have been looking at seeing a target from the reference frame
of the camera, and computing the horizontal distance (parallel to the playing field):
![alt text](./resources/camera_relative.jpg "target relative to limelight")

The next thing we need to do is convert from camera relative to robot relative, so we have a target distance
from the origin of the of the robot axis system to the target and a direction masured from the Y axis of the robot:
![alt text](./resources/robot_relative.jpg "target relative to robot")

Once we know where the target is relative to the robot, we need to use that information to determine the
position of the robot on the field. Note that we get the `robotHeading` from the NavX, and this let's
us get the angle from the target to the robot, and sice we have the distance, it should be easy to compute robot position on the field:
![alt text](./resources/field_relative.jpg "robot relative to the field")

This gives us a couple challenges:
* The robot may not always be facing the target so it is centered in the limelight screen, can we cope with
  that? Which really means we need to figure whether out our distance calculation is accurate when the target
  is off-center, and, if not, how do we we fix the distance calculation.
* The limelight is not on the center of the robot, how do we use the limelight offsets to compute the position
  of the robot center on the field.
* Currently the robot heading is also the limelight heading (though there could be a heading offset if the
  robot is modified). How does that factor into determining the field position of the robot?
  
Our first experiment was turning the robot so the limelight was not centered on the target.

In [9]:
double rot_target_x = Math.toRadians(-11.92);
double rot_target_y = Math.toRadians(10.39);
System.out.println("rot_target_x in radians = " + rot_target_x);
System.out.println("rot_target_y in radians = " + rot_target_y);
target_distance = getTargetDistance(rot_target_y);
target_distance

rot_target_x in radians = -0.20804324683772407
rot_target_y in radians = 0.18133970928221085


1.200997245634432

And, this distance was not at all close to the real measured distance - so, we need to go back and rethink things a bit. And I don't remember what the distance was that we measured.



### Dealing with Targets that are not Centered

We did a little experiment with rotating the robot and looking at the `target_delta`. It was not constant, but
rather, increased with either direction of rotation. What is going on here? We did a few thought experiments in
Aden's driveway with some cardboard representing target and plane of elevation of the limelight. We realized
that as the target moved off-center there was a change in the elevation angle - let's call this the
`actualElevation`. We built some 3D models which demonstrated that this `actualElevation` was a
function of the limelight X angle to the target:
![alt text](./resources/ElevatedLimelight.jpg "simple distance with limelight")

And then we did some derivations on cardboard and came to the formulas below. WHICH I THINK ARE WRONG, BUT I"M NOT SURE WHY !!!

In [9]:
double actualElevation(double x_angle)
{
    System.out.println("Math.cos(x_angle) = " + Math.cos(x_angle));
    return Math.sin(Math.sin(LIMELIGHT_ELEV_ANGLE) * Math.cos(x_angle));
}
double actualRotation(double x_angle)
{
    System.out.println("Math.cos(x_angle) = " + Math.cos(x_angle));
    return Math.atan2(Math.sin(x_angle),
                      Math.cos(LIMELIGHT_ELEV_ANGLE) * Math.cos(x_angle));
}

double actualElevation = actualElevation(rot_target_x);
System.out.println("LIMELIGHT_ELEV_ANGLE in radians = " + LIMELIGHT_ELEV_ANGLE);
System.out.println("actualElevation in radians = " + actualElevation);
double actualRotation = actualRotation(rot_target_x);
System.out.println("actualRotation in radians = " + actualRotation);
(TARGET_HEIGHT - LIMELIGHT_HEIGHT) / Math.tan(actualElevation + rot_target_y);

Math.cos(x_angle) = 0.978436946649733
LIMELIGHT_ELEV_ANGLE in radians = 0.337
actualElevation in radians = 0.3179128822882733
Math.cos(x_angle) = 0.978436946649733
actualRotation in radians = -0.22005713694458273


1.2561145824987987

### Dealing with the Limelight Offsets

This was a first try at using offsets to convert limelight position to robot position - but it was very niave and
cannot be easily generalized. This was our initial look at converting the camera distance for a target directly
in front of the camera to a robot distance and robot target angle - and the results where pretty accurate:

In [9]:
double y_dist = target_distance + LIMELIGHT_Y_OFFSET;
double x_dist = LIMELIGHT_X_OFFSET;
double robotTargetDistance = Math.sqrt((y_dist*y_dist) + (x_dist*x_dist));
robotTargetDistance

1.2000454492623522

In [10]:
double robotTargetAngle = Math.atan2(x_dist, y_dist);
Math.toDegrees(robotTargetAngle)

-9.593701436832506

**But** - after we did this, we started to realize that we needed way better math conventions going forward.